In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from glob import glob

# Path to the raw data directory
data_dir = '../data/raw/rural/RFID'

# Get all CSV files in the directory
csv_files = glob(os.path.join(data_dir, '*.CSV'))

# Dictionary to hold dataframes grouped by date
dataframes = {}

for file in csv_files:
    # Extract the base filename without extension
    basename = os.path.basename(file)
    name, _ = os.path.splitext(basename)
    
    # Remove suffix like _1 or _2 to get the date
    date_str = name.split('_')[0]
    
    # Read the CSV file
    df = pd.read_csv(file, sep=';')
    
    # Append the dataframe to the corresponding date group
    if date_str in dataframes:
        dataframes[date_str].append(df)
    else:
        dataframes[date_str] = [df]

# Combine dataframes for the same date
combined_dfs = []
for date, dfs in dataframes.items():
    combined_df = pd.concat(dfs, ignore_index=True)
    # Combine 'Date' and 'Time' into a single datetime column with dayfirst=True
    combined_df['Datetime'] = pd.to_datetime(
        combined_df['Date'] + ' ' + combined_df['Time'],
        dayfirst=True,
        format='%d-%m-%Y %H:%M:%S'
    )
    combined_dfs.append(combined_df)

# Concatenate all combined dataframes into a single dataframe
final_df = pd.concat(combined_dfs, ignore_index=True)

# Define period date ranges
periods = {
    1: ('2024-03-02', '2024-03-09'),
    2: ('2024-03-26', '2024-04-02'),
    3: ('2024-04-30', '2024-05-07'),
    4: ('2024-05-09', '2024-05-16'),
    5: ('2024-05-29', '2024-06-05')
}

# Initialize 'Period' column with NaN
final_df['Period'] = np.nan

# Assign period numbers based on 'Datetime'
for period, (start_date, end_date) in periods.items():
    mask = (final_df['Datetime'] >= start_date) & (final_df['Datetime'] <= end_date)
    final_df.loc[mask, 'Period'] = period

# Convert 'Period' column to integer type
final_df['Period'] = final_df['Period'].astype('Int64')

In [3]:
final_df['Transponder code'].unique()

array(['0007A3C016', '0007A3A179', '0007A01F42', '00079FC3C6',
       '0007A40088', '00079FEC60', '0007A3FDE2', '00079FEADE',
       '0007A3DD05', '0007A02701', '00079FEEA8', '0007A00464',
       '00079FED08', '0007A3C408', '0007A2E90C', '00079F8602',
       '0007A3DC62', '0007A3931B', '0007A2EFA1', '00079FC9A0',
       '3607A2E90C', '00079FCFF6', '01B79FEC60', '061FA02701',
       '0037A2EFA7', '0007A40CB8', '8000000001', '8000000003',
       '8000000002', '8000000000', '0187A3F31B', '00379FED38',
       '0000000000', '0007A3DD35', '00C7A03464', '000FA3DD45',
       '0C379FC9A0', '00079FEC66', '301FA3931B', '0037A3CC16',
       '00079BEC20', '00079FEDD0', '0006A3DC22', 8000000002, 8000000000,
       8000000003, 8000000001], dtype=object)

In [4]:
# Clean final_df by removing rows with NaN values in 'Period' column
clean_df = final_df.dropna(subset=['Period'])

# Define the list of desired transponder code suffixes
desired_suffixes = [
    '79FED08', '79FEADE', '7A3FDE2', '7A3931B', '7A2EFA1',
    '7A3DC62', '7A3C016', '7A3A179', '79FC3C6', '79FEC60',
    '7A02701', '7A2E90C', '7A3C408', '7A40088', '79FEEA8',
    '7A00464', '79F8602', '7A3DD05', '7A3E5BC'
]

# Filter the DataFrame to include only rows where 'Transponder code' starts with '000' and ends with the desired suffixes
filtered_df = clean_df[
    clean_df['Transponder code'].str.startswith('000') & 
    clean_df['Transponder code'].str.endswith(tuple(desired_suffixes))
].copy()  # Make an explicit copy to avoid SettingWithCopyWarning

# Shorten the Transponder codes by removing the first 3 characters
filtered_df.loc[:, 'Transponder code'] = filtered_df['Transponder code'].str[3:]

# Ensure the output directory exists
output_dir = '../data/intermediate-processing/rural'
os.makedirs(output_dir, exist_ok=True)

# Save the filtered DataFrame to a Parquet file
filtered_df.to_parquet(os.path.join(output_dir, 'RFID_filtered.parquet'), index=False)

In [7]:
filtered_df['Transponder code'].value_counts()

Transponder code
7A00464    373718
79FEEA8    329234
7A40088    309573
79FEC60    305927
7A02701    285169
7A3A179    258907
7A3C408    253691
7A3DD05    253488
7A3FDE2    246616
79FED08    207680
7A2EFA1    203027
79FEADE    183270
7A3DC62    181476
7A3931B    179823
79FC3C6    175915
7A3C016    165565
79F8602    164041
7A2E90C    134613
Name: count, dtype: int64

In [8]:
# Ensure the output directory exists
output_dir = '../data/intermediate-processing/rural'
os.makedirs(output_dir, exist_ok=True)

# Save the filtered DataFrame to a Parquet file
filtered_df.to_parquet(os.path.join(output_dir, 'RFID_filtered.parquet'), index=False)